###**Welcome to this tutorial where we will show you how to replicate the results from the paper titled “Fairness-Aware Structured Pruning in Transformers”**###


# Creating the script files to launch the experiments

In [1]:
from google.colab import drive
# As an example, I am using "cd /content/drive/MyDrive/EMNLP_reproducibility" as my directory
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/PhD/reproducibility/FASP

/content/drive/MyDrive/PhD/reproducibility/FASP


In [ ]:
!unzip -o FASP_github.zip

Archive:  FASP_github.zip
  inflating: collect_results.py      
  inflating: head_scores.py          
  inflating: main.py                 
  inflating: merge_chunks.py         
   creating: model/
   creating: model/.ipynb_checkpoints/
  inflating: model/data_loader.py    
  inflating: model/generation.py     
  inflating: model/head_contributions.json  
  inflating: model/head_scores_distilgpt2.csv  
  inflating: model/head_scores_distilgpt2_other_biases.csv  
  inflating: model/head_scores_EleutherAI_gpt-neo-1.3B_abdel1.csv  
  inflating: model/head_scores_EleutherAI_gpt-neo-1.3B_olewicki.csv  
  inflating: model/head_scores_EleutherAI_gpt-neo-125M.csv  
  inflating: model/head_scores_EleutherAI_gpt-neo-125M_other_biases.csv  
  inflating: model/head_scores_facebook_opt-350m.csv  
  inflating: model/head_scores_gpt2.csv  
  inflating: model/head_scores_gpt2_other_biases.csv  
  inflating: model/wikitext-2-raw-v1.txt  
  inflating: model/wikitext-2-raw-v1_test.txt  
  inflating: mode

### Computing the performance and bias head scores

This generates 11,160 experiments overall

In [ ]:
import numpy as np
from os import getgroups
from pathlib import Path

id = 0
#,"nationality", "race_ethnicity", "religion", "sexual_orientation", "gender_and_sex"
#"EleutherAI/gpt-neo-1.3B","EleutherAI/gpt-neo-125M","gpt2","distilgpt2", "meta-llama/Llama-2-7b-chat-hf"
folder_name = "experiment_17"
heads_per_sh = 10
for group in ["gender_and_sex"]:
  for model in ["meta-llama/Llama-2-7b-chat-hf"]:

      if model == "EleutherAI/gpt-neo-1.3B":
        num_all_heads = 384
      elif model == "EleutherAI/gpt-neo-125M":
        num_all_heads = 144
      elif model == "EleutherAI/gpt-j-6B":
        num_all_heads = 448
      elif model == "meta-llama/Llama-2-7b-chat-hf":
        num_all_heads = 1024
      elif model == "gpt2":
        num_all_heads = 144
      elif model == "distilgpt2":
        num_all_heads = 72

      for prompting in ["holistic"]:
        for head_knockout in range(0,num_all_heads,heads_per_sh):
          for seed in range(1,4):
            my_file = open("./scripts/sample.sh")
            string_list = my_file.readlines()

            my_file.close()

            # string_list[1] = "#SBATCH --account=rrg-bengioy-ad_gpu" + "\n"
            # string_list[1] = "#SBATCH --account=rrg-bengioy-ad" + "\n"
            string_list[1] = "#SBATCH --account=def-bengioy_cpu" + "\n"
            string_list[2] = "#SBATCH --cpus-per-task=1" + "\n"
            string_list[3] =  "\n"
            string_list[4] = "#SBATCH --mem=25G" + "\n"
            string_list[8] = "python compute_toxicity.py "

            if model == "EleutherAI/gpt-neo-125M":
              model_name = "N1"
              string_list[4] = "#SBATCH --mem=50G" + "\n"

            elif model == "EleutherAI/gpt-j-6B":
              model_name = "NJ"
              string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
              string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
              string_list[4] = "#SBATCH --mem=100G" + "\n"
              string_list[8] += " --batch_size 64 "

            elif model == "meta-llama/Llama-2-7b-chat-hf":
              model_name = "L7"
              string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
              string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
              string_list[4] = "#SBATCH --mem=100G" + "\n"
              string_list[8] += " --batch_size 64 "

            elif model == "EleutherAI/gpt-neo-1.3B":
              model_name = "N2"
              string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
              string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
              string_list[4] = "#SBATCH --mem=50G" + "\n"
              string_list[8] += " --batch_size 128 "

            elif model == "EleutherAI/gpt-neo-2.7B":
              model_name = "N3"
              string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
              string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
              string_list[4] = "#SBATCH --mem=50G" + "\n"
              string_list[8] += " --batch_size 128 "

            elif model == "distilgpt2":
              model_name = "G2D"

            elif model == "gpt2":
              model_name = "G2"

            elif model == "gpt2-medium":
              model_name = "G2M"

            elif model == "gpt2-large":
              model_name = "G2L"

            elif model == "bert-base-cased":
              model_name = "BB"

            elif model == "bert-large-cased":
              model_name = "BL"

            elif model == "roberta-base":
              model_name = "RB"

            elif model == "roberta-large":
              model_name = "RL"

            elif model == "distilroberta-base":
              model_name = "RD"

            elif model == "distilbert-base-cased":
              model_name = "BD"

            elif model == "EleutherAI/pythia-160m":
              model_name = "P2"

            elif model == "facebook/opt-350m":
              model_name = "O1"

            string_list[5] = "#SBATCH --time=11:57:00" + "\n"

            if group == "gender_and_sex":
              group_name = "g"
            elif group == "race_ethnicity":
              group_name = "r"
            elif group == "religion":
              group_name = "l"
            elif group == "sexual_orientation":
              group_name = "s"
            elif group == "nationality":
              group_name = "n"

            string_list[8] += " --model " + model

            prompting_abbrev="h"
            string_list[8] += " --targeted_holistic_bias " + group

            string_list[8] += " --prompting " + str(prompting)

            string_list[7] = "sleep " + str(1*id) + "\n"
            file_name = "./scripts/" + folder_name  + "/" + str(prompting_abbrev) + str(seed) + "_" + str(model_name) + "_h" + str(head_knockout) + "_"  + str(group_name)
            string_list[8] = string_list[8] + " --seed " + str(seed)
            temp = string_list[8]
            string_list[8] = temp + " --head_knockout " + str(head_knockout) + "\n"
            for h in range(1,heads_per_sh):
              string_list[8] += temp + " --head_knockout " + str(head_knockout + h) + "\n"
            Path("./scripts/" + folder_name).mkdir(parents=True, exist_ok=True)
            my_file = open(file_name + ".sh", "w")
            new_file_contents = "".join(string_list)
            my_file.write(new_file_contents)
            my_file.close()
          id+=1

### Comparing different pruning methods (small models that don't need chunking of the experimets)
This generates overall 7,260 experiments

In [ ]:
import numpy as np
from os import getgroups
from pathlib import Path

id = 0
#,"race_ethnicity", "religion", "sexual_orientation", "nationality", "gender_and_sex"
#"EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","gpt2","distilgpt2", "meta-llama/Llama-2-7b-chat-hf"
#meta-llama/Llama-2-7b-chat-hf
#,"FASP"
#"FASP", "bias_only", "ppl_only","random_structured", "mask_gradient_l2_structured", "magnitude_l2_structured", "bias_ppl"
#"0.2", "0.3","0.4","0.5","0.6", "0.7"
folder_name = "experiment_29"
for group in ["gender_and_sex"]:
  for model in ["EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","gpt2","distilgpt2"]:
    for pruned_heads_ratio in np.linspace(0,0.2,11,endpoint=True):
      for prompting in ["holistic"]:
        for method in ["bias_ppl"]:
          if method == "FASP":
            gammas = ["0.2", "0.3","0.4","0.5","0.6", "0.7"]
          else:
            gammas = ["None"]
          if method == "bias_ppl":
            betas = ["0.2", "0.3","0.4","0.5","0.6", "0.7"]
          else:
            betas = ["None"]
          for gamma in gammas:
            for beta in betas:
              for seed in range(1,4):
                my_file = open("./scripts/sample.sh")
                string_list = my_file.readlines()

                my_file.close()
                # string_list[1] = "#SBATCH --account=rrg-bengioy-ad_gpu" + "\n"
                # string_list[1] = "#SBATCH --account=rrg-bengioy-ad" + "\n"
                string_list[1] = "#SBATCH --account=def-bengioy_cpu" + "\n"
                string_list[2] = "#SBATCH --cpus-per-task=1" + "\n"
                string_list[3] =  "\n"
                string_list[4] = "#SBATCH --mem=25G" + "\n"
                string_list[8] = "python main.py "
                # string_list[8] = "python compute_toxicity.py "

                if model == "EleutherAI/gpt-neo-125M":
                  model_name = "N1"
                  string_list[4] = "#SBATCH --mem=50G" + "\n"

                elif model == "EleutherAI/gpt-neo-1.3B":
                  model_name = "N2"
                  string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                  string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                  string_list[4] = "#SBATCH --mem=50G" + "\n"
                  string_list[8] += " --batch_size 128 "

                elif model == "EleutherAI/gpt-j-6B":
                  model_name = "NJ"
                  string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                  string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                  string_list[4] = "#SBATCH --mem=100G" + "\n"
                  string_list[8] += " --batch_size 64 "

                elif model == "meta-llama/Llama-2-7b-chat-hf":
                  model_name = "L7"
                  string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                  # string_list[1] = "#SBATCH --account=def-bengioy" + "\n"
                  string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                  # string_list[4] = "#SBATCH --mem=50G" + "\n"
                  string_list[4] = "#SBATCH --mem=100G" + "\n"
                  string_list[8] += " --batch_size 64 "

                elif model == "EleutherAI/gpt-neo-2.7B":
                  model_name = "N3"
                  string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                  string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                  string_list[4] = "#SBATCH --mem=50G" + "\n"
                  string_list[8] += " --batch_size 128 "

                elif model == "distilgpt2":
                  model_name = "G2D"

                elif model == "gpt2":
                  model_name = "G2"

                elif model == "gpt2-medium":
                  model_name = "G2M"

                elif model == "gpt2-large":
                  model_name = "G2L"

                elif model == "bert-base-cased":
                  model_name = "BB"

                elif model == "bert-large-cased":
                  model_name = "BL"

                elif model == "roberta-base":
                  model_name = "RB"

                elif model == "roberta-large":
                  model_name = "RL"

                elif model == "distilroberta-base":
                  model_name = "RD"

                elif model == "distilbert-base-cased":
                  model_name = "BD"

                elif model == "EleutherAI/pythia-160m":
                  model_name = "P2"

                elif model == "facebook/opt-350m":
                  model_name = "O1"

                string_list[5] = "#SBATCH --time=11:57:00" + "\n"

                if group == "gender_and_sex":
                  group_name = "g"
                elif group == "race_ethnicity":
                  group_name = "r"
                elif group == "religion":
                  group_name = "l"
                elif group == "sexual_orientation":
                  group_name = "s"
                elif group == "nationality":
                  group_name = "n"

                string_list[8] += " --model " + model
                string_list[8] += " --method " + str(method)
                string_list[8] += " --pruned_heads_ratio " + str(pruned_heads_ratio)
                if gamma != "None":
                  string_list[8] += " --gamma " + str(gamma)
                if beta != "None":
                  string_list[8] += " --beta " + str(beta)

                if prompting == "holistic":
                  prompting_abbrev="h"
                  string_list[8] += " --targeted_holistic_bias " + group

                if method == "random_structured":
                  method_name = "rn"
                elif method == "FASP":
                  method_name = "f"
                elif method == "bias_ppl":
                  method_name = "bp"
                elif method == "magnitude_l1_structured":
                  method_name = "m1"
                elif method == "magnitude_l2_structured":
                  method_name = "m2"
                elif method == "magnitude_linf_structured":
                  method_name = "mf"
                elif method == "mask_gradient_l2_structured":
                  method_name = "mg2"
                elif method == "bias_only":
                  method_name = "bo"
                elif method == "ppl_only":
                  method_name = "po"

                string_list[8] += " --prompting " + str(prompting)
                string_list[7] = "sleep " + str(1*id) + "\n"
                file_name = "./scripts/" + folder_name  + "/" + str(prompting_abbrev) + str(seed) + "_" + str(model_name) + "_" + str(method_name) + "_"  + str(pruned_heads_ratio) + "_"  + str(group_name)

                if method == "FASP":
                  file_name += "_" + str(gamma)
                if method == "bias_ppl":
                  file_name += "_" + str(beta)

                string_list[8] += " --seed " + str(seed) + " \n"
                Path("./scripts/" + folder_name).mkdir(parents=True, exist_ok=True)
                my_file = open(file_name + ".sh", "w")
                new_file_contents = "".join(string_list)
                my_file.write(new_file_contents)
                my_file.close()
              id+=1

### Combining pruning methods with debiasing (small models that don't need chunking of the experimets)
This generates overall 924 experiments

In [ ]:
import numpy as np
from os import getgroups
from pathlib import Path

id = 0
#,"race_ethnicity", "religion", "sexual_orientation", "nationality", "gender_and_sex"
#"EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","gpt2","distilgpt2", "meta-llama/Llama-2-7b-chat-hf"
#meta-llama/Llama-2-7b-chat-hf
#,"FASP"
#"FASP", "bias_only", "ppl_only","random_structured", "mask_gradient_l2_structured", "magnitude_l2_structured", "bias_ppl"
#"0.2", "0.3","0.4","0.5","0.6", "0.7"
#, "temperature_scaling", "random_perturbation"
gamma, beta = "None", "None"
folder_name = "experiment_37"
for group in ["gender_and_sex"]:
  for model in ["EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","gpt2","distilgpt2"]:
    for pruned_heads_ratio in np.linspace(0,0.2,11,endpoint=True):
      for prompting in ["holistic"]:
        for method in ["ppl_only"]:
          for debiasing in ["temperature_scaling", "random_perturbation"]:
              if debiasing == "temperature_scaling":
                att_scaling_factors = ["0.7", "0.8", "0.9","1", "1.1", "1.2", "1.3"]
              else:
                att_scaling_factors = ["None"]

              for att_scaling_factor in att_scaling_factors:
                  for seed in range(1,4):
                    my_file = open("./scripts/sample.sh")
                    string_list = my_file.readlines()

                    my_file.close()
                    # string_list[1] = "#SBATCH --account=rrg-bengioy-ad_gpu" + "\n"
                    # string_list[1] = "#SBATCH --account=rrg-bengioy-ad" + "\n"
                    string_list[1] = "#SBATCH --account=def-bengioy_cpu" + "\n"
                    string_list[2] = "#SBATCH --cpus-per-task=1" + "\n"
                    string_list[3] =  "\n"
                    string_list[4] = "#SBATCH --mem=25G" + "\n"
                    string_list[8] = "python main.py "
                    # string_list[8] = "python compute_toxicity.py "

                    if model == "EleutherAI/gpt-neo-125M":
                      model_name = "N1"
                      string_list[4] = "#SBATCH --mem=50G" + "\n"

                    elif model == "EleutherAI/gpt-neo-1.3B":
                      model_name = "N2"
                      string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                      string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                      string_list[4] = "#SBATCH --mem=50G" + "\n"
                      string_list[8] += " --batch_size 128 "

                    elif model == "EleutherAI/gpt-j-6B":
                      model_name = "NJ"
                      string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                      string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                      string_list[4] = "#SBATCH --mem=100G" + "\n"
                      string_list[8] += " --batch_size 64 "

                    elif model == "meta-llama/Llama-2-7b-chat-hf":
                      model_name = "L7"
                      string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                      # string_list[1] = "#SBATCH --account=def-bengioy" + "\n"
                      string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                      # string_list[4] = "#SBATCH --mem=50G" + "\n"
                      string_list[4] = "#SBATCH --mem=100G" + "\n"
                      string_list[8] += " --batch_size 64 "

                    elif model == "EleutherAI/gpt-neo-2.7B":
                      model_name = "N3"
                      string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                      string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                      string_list[4] = "#SBATCH --mem=50G" + "\n"
                      string_list[8] += " --batch_size 128 "

                    elif model == "distilgpt2":
                      model_name = "G2D"

                    elif model == "gpt2":
                      model_name = "G2"

                    elif model == "gpt2-medium":
                      model_name = "G2M"

                    elif model == "gpt2-large":
                      model_name = "G2L"

                    elif model == "bert-base-cased":
                      model_name = "BB"

                    elif model == "bert-large-cased":
                      model_name = "BL"

                    elif model == "roberta-base":
                      model_name = "RB"

                    elif model == "roberta-large":
                      model_name = "RL"

                    elif model == "distilroberta-base":
                      model_name = "RD"

                    elif model == "distilbert-base-cased":
                      model_name = "BD"

                    elif model == "EleutherAI/pythia-160m":
                      model_name = "P2"

                    elif model == "facebook/opt-350m":
                      model_name = "O1"

                    string_list[5] = "#SBATCH --time=11:57:00" + "\n"

                    if group == "gender_and_sex":
                      group_name = "g"
                    elif group == "race_ethnicity":
                      group_name = "r"
                    elif group == "religion":
                      group_name = "l"
                    elif group == "sexual_orientation":
                      group_name = "s"
                    elif group == "nationality":
                      group_name = "n"

                    string_list[8] += " --model " + model
                    string_list[8] += " --method " + str(method)
                    if debiasing != "None":
                      string_list[8] += " --intraprocessing_method " + str(debiasing)

                    string_list[8] += " --pruned_heads_ratio " + str(pruned_heads_ratio)
                    if gamma != "None":
                      string_list[8] += " --gamma " + str(gamma)
                    if beta != "None":
                      string_list[8] += " --beta " + str(beta)
                    if att_scaling_factor != "None":
                      string_list[8] += " --attn_scale " + str(att_scaling_factor)

                    if prompting == "holistic":
                      prompting_abbrev="h"
                      string_list[8] += " --targeted_holistic_bias " + group

                    if method == "random_structured":
                      method_name = "rn"
                    elif method == "FASP":
                      method_name = "f"
                    elif method == "bias_ppl":
                      method_name = "bp"
                    elif method == "magnitude_l1_structured":
                      method_name = "m1"
                    elif method == "magnitude_l2_structured":
                      method_name = "m2"
                    elif method == "magnitude_linf_structured":
                      method_name = "mf"
                    elif method == "mask_gradient_l2_structured":
                      method_name = "mg2"
                    elif method == "bias_only":
                      method_name = "bo"
                    elif method == "ppl_only":
                      method_name = "po"

                    if debiasing == "temperature_scaling":
                      debiasing_name = "t"
                    elif debiasing == "random_perturbation":
                      debiasing_name = "r"

                    string_list[8] += " --prompting " + str(prompting)
                    string_list[7] = "sleep " + str(1*id) + "\n"
                    file_name = "./scripts/" + folder_name  + "/" + str(prompting_abbrev) + str(seed) + "_" + str(model_name) + "_" + str(method_name) + "_"  + str(pruned_heads_ratio) + "_"  + str(group_name)
                    if debiasing != "None":
                      file_name += str(debiasing_name)
                    if att_scaling_factor != "None":
                      file_name += str(att_scaling_factor)

                    if method == "FASP":
                      file_name += "_" + str(gamma)
                    if method == "bias_ppl":
                      file_name += "_" + str(beta)

                    string_list[8] += " --seed " + str(seed) + " \n"
                    Path("./scripts/" + folder_name).mkdir(parents=True, exist_ok=True)
                    my_file = open(file_name + ".sh", "w")
                    new_file_contents = "".join(string_list)
                    my_file.write(new_file_contents)
                    my_file.close()
                  id+=1

### Comparing different pruning methods (large models that need chunking of the experimets)
This generates overall 7,260 experiments

In [ ]:
import numpy as np
from os import getgroups

id = 0
for chunk_id in range(1,6):
  for group in ["gender_and_sex","race_ethnicity", "religion", "sexual_orientation", "nationality"]:
    for model in ["EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","gpt2","distilgpt2"]:
      for pruned_heads_ratio in np.linspace(0,0.2,11,endpoint=True):
        for prompting in ["holistic"]:
          for method in ["bias_only", "ppl_only","random_structured", "mask_gradient_l2_structured", "magnitude_l2_structured","FASP"]:
            if method == "FASP":
              gammas = ["0.2", "0.3","0.4","0.5","0.6", "0.7"]
            else:
              gammas = ["None"]
            for gamma in gammas:
              for seed in range(1,4):
                my_file = open("./scripts/sample.sh")
                string_list = my_file.readlines()

                my_file.close()
                # string_list[1] = "#SBATCH --account=rrg-bengioy-ad_gpu" + "\n"
                # string_list[1] = "#SBATCH --account=rrg-bengioy-ad" + "\n"
                string_list[1] = "#SBATCH --account=def-bengioy_cpu" + "\n"
                string_list[2] = "#SBATCH --cpus-per-task=1" + "\n"
                string_list[3] =  "\n"
                string_list[4] = "#SBATCH --mem=25G" + "\n"
                string_list[8] = "python main.py "

                if model == "EleutherAI/gpt-neo-125M":
                  model_name = "N1"
                  string_list[4] = "#SBATCH --mem=50G" + "\n"

                elif model == "EleutherAI/gpt-neo-1.3B":
                  model_name = "N2"
                  string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                  string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                  string_list[4] = "#SBATCH --mem=50G" + "\n"
                  string_list[8] += " --batch_size 128 "

                elif model == "EleutherAI/gpt-neo-2.7B":
                  model_name = "N3"
                  string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
                  string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                  string_list[4] = "#SBATCH --mem=50G" + "\n"
                  string_list[8] += " --batch_size 128 "

                elif model == "distilgpt2":
                  model_name = "G2D"

                elif model == "gpt2":
                  model_name = "G2"

                elif model == "gpt2-medium":
                  model_name = "G2M"

                elif model == "gpt2-large":
                  model_name = "G2L"

                elif model == "bert-base-cased":
                  model_name = "BB"

                elif model == "bert-large-cased":
                  model_name = "BL"

                elif model == "roberta-base":
                  model_name = "RB"

                elif model == "roberta-large":
                  model_name = "RL"

                elif model == "distilroberta-base":
                  model_name = "RD"

                elif model == "distilbert-base-cased":
                  model_name = "BD"

                elif model == "EleutherAI/pythia-160m":
                  model_name = "P2"

                elif model == "facebook/opt-350m":
                  model_name = "O1"

                string_list[5] = "#SBATCH --time=11:57:00" + "\n"

                if group == "gender_and_sex":
                  group_name = "g"
                elif group == "race_ethnicity":
                  group_name = "r"
                elif group == "religion":
                  group_name = "l"
                elif group == "sexual_orientation":
                  group_name = "s"
                elif group == "nationality":
                  group_name = "n"

                string_list[8] += " --model " + model
                string_list[8] += " --method " + str(method)
                string_list[8] += " --pruned_heads_ratio " + str(pruned_heads_ratio)
                if gamma != "None":
                  string_list[8] += " --gamma " + str(gamma)

                if prompting == "holistic":
                  prompting_abbrev="h"
                  string_list[8] += " --targeted_holistic_bias " + group

                if method == "random_structured":
                  method_name = "rn"
                elif method == "FASP":
                  method_name = "f"
                elif method == "magnitude_l1_structured":
                  method_name = "m1"
                elif method == "magnitude_l2_structured":
                  method_name = "m2"
                elif method == "magnitude_linf_structured":
                  method_name = "mf"
                elif method == "mask_gradient_l2_structured":
                  method_name = "mg2"
                elif method == "bias_only":
                  method_name = "bo"
                elif method == "ppl_only":
                  method_name = "po"

                string_list[8] += " --prompting " + str(prompting)
                string_list[7] = "sleep " + str(1*id) + "\n"
                file_name = "./scripts/" + str(prompting_abbrev) + str(seed) + "_" + str(model_name) + "_" + str(method_name) + "_"  + str(pruned_heads_ratio) + "_"  + str(group_name)

                file_name += str(chunk_id)
                string_list[8] += " --chunk_id " + str(chunk_id)
                if method == "FASP":
                  file_name += "_" + str(gamma)

                string_list[8] += " --seed " + str(seed) + " \n"
                my_file = open(file_name + ".sh", "w")
                new_file_contents = "".join(string_list)
                my_file.write(new_file_contents)
                my_file.close()
              id+=1

# Running the code:

In [ ]:
cd /content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work

/content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work


In [ ]:
!pip install -r requirements.txt

In [ ]:
!python main.py  --batch_size 64  --model meta-llama/Llama-2-7b-chat-hf --method random_structured --pruned_heads_ratio 0.18 --targeted_holistic_bias gender_and_sex --prompting holistic --seed 3

Traceback (most recent call last):
  File "/content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work/main.py", line 8, in <module>
    from transformers_newest import AutoTokenizer, BloomTokenizerFast, GPT2Tokenizer
  File "/content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work/transformers_newest/__init__.py", line 26, in <module>
    from . import dependency_versions_check
  File "/content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work/transformers_newest/dependency_versions_check.py", line 16, in <module>
    from .utils.versions import require_version, require_version_core
  File "/content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work/transformers_newest/utils/__init__.py", line 61, in <module>
    from .hub import (
  File "/content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work/transformers_newest/utils/hub.py", line 43, in <module>
    from huggingface_hub.utils import (
ImportError: cannot import name 'GatedRepoError' from 'huggingfa

# Pre-processing

## Creating configuration file for models

In [ ]:
from transformers import AutoModelWithLMHead,AutoTokenizer
import torch
from configparser import ConfigParser
from transformers import AutoTokenizer, BloomTokenizerFast, GPT2Tokenizer, GPTNeoForCausalLM, GPTNeoXForCausalLM, GPTNeoForCausalLM, GPTNeoXTokenizerFast, BloomForCausalLM, AutoTokenizer, AutoModelWithLMHead, BloomTokenizerFast, OPTForCausalLM, GPT2Tokenizer, GPTJForCausalLM
import json, torch
#"gpt2", "EleutherAI/gpt-neo-1.3B"
head_contributions = {}
model_names = ["distilgpt2", "EleutherAI/gpt-neo-125M", "gpt2", "EleutherAI/gpt-neo-1.3B"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models_config = {}
# json.dump(model_configs, open("./model/models_config.json", "w"))

for model_name in model_names:

    #Get the configparser object
    # model_configs = json.load(open("./model/models_config.json", "r"))
    # models_config = {}
    models_config[model_name] = {}

    if model_name in ["gpt2", "gpt2-medium", "gpt2-large", "distilgpt2",  "gpt2-xl"]:
        model = AutoModelWithLMHead.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")   # Initialize tokenizer
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions

    elif model_name in ["distilroberta-base", "distilbert-base-cased", "bert-base-cased",  "bert-large-cased", "roberta-base","roberta-large"]:
        model = AutoModelWithLMHead.from_pretrained("./saved_models/cached_models/" + model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")   # Initialize tokenizer
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["EleutherAI/gpt-neo-125M", "EleutherAI/gpt-neo-1.3B", "EleutherAI/gpt-neo-2.7B"]:
        model = GPTNeoForCausalLM.from_pretrained(model_name).to(device)
        tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side="left")
        num_heads, num_layers = model.config.num_heads, model.config.num_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings
        # for name, para in model.named_parameters():
            # print(name, para.shape)

    elif model_name in ["EleutherAI/gpt-j-6B"]:
        model = GPTJForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name, revision="float16",torch_dtype=torch.float16,).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions
        # for name, para in model.named_parameters():
            # print(name, para.shape)

    elif model_name in ["facebook/opt-350m", "facebook/opt-1.3b", "facebook/opt-2.7b", "facebook/opt-6.7b"]:
        model = OPTForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["bigscience/bloom-560m", "bigscience/bloom-1b1","bigscience/bloom-3b", "bigscience/bloom-7b1"]:
        model = BloomForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name).to(device)
        tokenizer = BloomTokenizerFast.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        num_heads, num_layers = model.config.num_attention_heads, model.config.n_layer
        head_dim, max_length = int(model.config.hidden_size/num_heads), 1024

    elif model_name in ["EleutherAI/pythia-70m","EleutherAI/pythia-160m","EleutherAI/pythia-410m","EleutherAI/pythia-1b","EleutherAI/pythia-1.4b","EleutherAI/pythia-2.8b","EleutherAI/pythia-6.9b","EleutherAI/pythia-12b"]:
        model = GPTNeoXForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["meta-llama/Llama-2-7b"]:
        model = LlamaForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name, revision="float16",torch_dtype=torch.float16,).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.params.n_heads, model.params.n_layers
        head_dim, max_length = int(model.params.dim/num_heads), 1024

    #Assume we need 2 sections in the config file, let's call them USERINFO and SERVERCONFIG
    models_config[model_name]["num_heads"] = num_heads
    models_config[model_name]["num_layers"] = num_layers
    models_config[model_name]["head_dim"] = head_dim
    models_config[model_name]["max_length"] = max_length

json.dump(models_config, open("./model/models_config.json", "w"))

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

## Downloading models

In [ ]:
from transformers import AutoModelWithLMHead,AutoTokenizer
import torch
from transformers import AutoTokenizer, BloomTokenizerFast, GPT2Tokenizer, LlamaForCausalLM, GPTNeoForCausalLM, GPTNeoXForCausalLM, GPTNeoForCausalLM, GPTNeoXTokenizerFast, BloomForCausalLM, AutoTokenizer, AutoModelWithLMHead, BloomTokenizerFast, OPTForCausalLM, GPT2Tokenizer, GPTJForCausalLM

# This file used to download the models from huggingface and save them in the cached_models folder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#"EleutherAI/pythia-70m","EleutherAI/pythia-160m","EleutherAI/pythia-410m","EleutherAI/pythia-1b","EleutherAI/pythia-1.4b","EleutherAI/gpt-neo-125M", "EleutherAI/gpt-neo-1.3B", "facebook/opt-350m", "facebook/opt-1.3b","bigscience/bloom-560m", "bigscience/bloom-1b1","gpt2", "gpt2-medium", "gpt2-large","distilgpt2", "bert-base-cased",  "bert-large-cased", "roberta-base","roberta-large"
for model_name in ["meta-llama/Llama-2-7b-chat-hf"]:
    print(model_name)
    if model_name in ["gpt2", "gpt2-medium", "gpt2-large", "distilgpt2",  "gpt2-xl"]:
        model = AutoModelWithLMHead.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")   # Initialize tokenizer
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions
    elif model_name in ["distilroberta-base", "distilbert-base-cased", "bert-base-cased",  "bert-large-cased", "roberta-base","roberta-large"]:
        model = AutoModelWithLMHead.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")   # Initialize tokenizer
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["EleutherAI/gpt-neo-125M", "EleutherAI/gpt-neo-1.3B", "EleutherAI/gpt-neo-2.7B"]:
        model = GPTNeoForCausalLM.from_pretrained(model_name).to(device)
        tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side="left")
        num_heads, num_layers = model.config.num_heads, model.config.num_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["EleutherAI/gpt-j-6B"]:
        model =  GPTJForCausalLM.from_pretrained(model_name,revision="float16", torch_dtype=torch.float16,).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side="left")
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions

    elif model_name in ["EleutherAI/gpt-neox-20b"]:
        model = GPTNeoXForCausalLM.from_pretrained(model_name)
        tokenizer = GPTNeoXTokenizerFast.from_pretrained(model_name)
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["facebook/opt-350m", "facebook/opt-1.3b", "facebook/opt-2.7b", "facebook/opt-6.7b"]:
        model = OPTForCausalLM.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["bigscience/bloom-560m", "bigscience/bloom-1b1","bigscience/bloom-3b", "bigscience/bloom-7b1"]:
        model = BloomForCausalLM.from_pretrained(model_name).to(device)
        tokenizer = BloomTokenizerFast.from_pretrained(model_name, padding_side="left")
        num_heads, num_layers = model.config.num_attention_heads, model.config.n_layer
        head_dim, max_length = int(model.config.hidden_size/num_heads), 1024

    elif model_name in ["EleutherAI/pythia-70m","EleutherAI/pythia-160m","EleutherAI/pythia-410m","EleutherAI/pythia-1b","EleutherAI/pythia-1.4b","EleutherAI/pythia-2.8b","EleutherAI/pythia-6.9b","EleutherAI/pythia-12b"]:
        model = GPTNeoXForCausalLM.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings


    elif model_name in ["meta-llama/Llama-2-7b-chat-hf"]:
        model = LlamaForCausalLM.from_pretrained(model_name, revision="float16",torch_dtype=torch.float16,).to(device)
        tokenizer = AutoTokenizer.from_pretrained( model_name, padding_side="left")
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.dim/num_heads), model.config.max_position_embeddings


    model.save_pretrained("./saved_models/cached_models/" + model_name)
    tokenizer.save_pretrained("./saved_models/cached_tokenizers/" + model_name)


In [ ]:


from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

'''
uncomment to get reproducable paraphrase generations
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)
'''

#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

phrases = ["Can you recommend some upscale restaurants in Newyork?",
           "What are the famous places we should not miss in Russia?"
]

for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  para_phrases = parrot.augment(input_phrase=phrase, use_gpu=False)
  for para_phrase in para_phrases:
   print(para_phrase)

ModuleNotFoundError: ignored

In [ ]:
pip install openai

In [ ]:
!pip install transformers

!pip install torch

!pip install sentencepiece

!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=6eca7fc02a5abf5cd1ef2691cb52ba6a00a64fa9aa9ec1a7271d511f68683bf4
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3339 sha256=0657f40922c8f1838df4546818aa12030e2952f35edccbd17c8fd69cfa08f138
  Store

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws").to('cuda')

sentence = "This is something which i cannot understand at all"

text =  "paraphrase: " + sentence + " </s>"

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")


outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    do_sample=True,
    top_k=120,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=5
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(line)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


This is something that i can not understand at all.
This is something i cannot understand at all.
I cannot even understand this
This is something i can hardly understand at all.
This is something which i can't understand at all.
